<a href="https://colab.research.google.com/github/codeson101001/Stock-Prediction-Models/blob/master/testing_multi_stocks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [71]:
  import numpy as np
  import pandas as pd
  import matplotlib.pyplot as plt
  import seaborn as sns
  sns.set()
  !pip install yfinance
  import yfinance as yf 

In [72]:
testing_multiple_stocks()

def testing_multiple_stocks():

  starting = "2019-01-01"
  ending = "2019-06-28"

  stock_strategy("AAPL",starting,ending)
  stock_strategy("AMD",starting,ending)
  stock_strategy("AMZN",starting,ending)
  stock_strategy("GOOG",starting,ending)
  stock_strategy("MSFT",starting,ending)
  stock_strategy("TSLA",starting,ending)
  stock_strategy("NVDA",starting,ending)
  stock_strategy("T",starting,ending)

AAPL
[*********************100%***********************]  1 of 1 completed
day 5: buy 260 units at price 9965.149841, total balance 34.850159
day 9, sell 260 units at price 9949.550476, investment -0.156539 %, total balance 9984.400635,
day 10: buy 257 units at price 9954.895157, total balance 29.505478
day 14, sell 257 units at price 9889.359882, investment -0.658322 %, total balance 9918.865360,
day 16: buy 251 units at price 9899.439655, total balance 19.425705
day 27, sell 251 units at price 10631.732040, investment 7.397311 %, total balance 10651.157745,
day 31: buy 249 units at price 10608.644886, total balance 42.512859
day 44, sell 249 units at price 10738.125000, investment 1.220515 %, total balance 10780.637859,
day 46: buy 241 units at price 10778.724632, total balance 1.913227
day 57, sell 241 units at price 11254.097095, investment 4.410285 %, total balance 11256.010323,
day 61: buy 235 units at price 11235.350323, total balance 20.660000
day 71, sell 235 units at price 117

In [73]:
def stock_strategy(stock,starting_date,ending_date):
  print(stock)

  #df = yf.download(tickers="MMM", period="1y", interval="1d")  
  df = yf.download(stock, start = starting_date, end = ending_date)


  #This is how I calucalte my short and long window for the moving avg in the future
  short_window = int(0.020 * len(df))
  long_window = int(0.06 * len(df))

  #We are storing the data from the stock information into signals
  signals = pd.DataFrame(index = df.index)

  #Here we are creating our dataframe column titles
  signals['price'] = df['Close']

  #We start by seting signals to 0.0 
  signals['signal'] = 0.0
  signals['short_ma'] = df['Close'].rolling(window=short_window, min_periods=1, center=False).mean()
  signals['long_ma'] = df['Close'].rolling(window=long_window, min_periods=1, center=False).mean()

  signals['signal'][short_window:] = np.where(signals['short_ma'][short_window:] > signals['long_ma'][short_window:], 1.0, 0.0)   
  signals['positions'] = signals['signal'].diff()

  signals







  #This is the stock simulator
  def buy_stock(real_movement, signal, initial_money = 10000, max_buy = 3500, max_sell = 3500):

      starting_money = initial_money
      states_sell = []
      states_buy = []
      current_inventory = 0

      #"i" is the day
      #"initial_money" is your money that you started out with
      #"real_movement" is the your money in the stock, which can be larger or smaller that the initial_money 
      def buy(i, initial_money, current_inventory):
          #Here we calualate the number of shares we want to buy using floor division, beucase we can not get pieces of a stock share
          shares = initial_money // real_movement[i]

          if shares < 1:
              print('day %d: total balances %f, not enough money to buy a unit price %f'% (i, initial_money, real_movement[i]))

          else:

              if shares > max_buy:
                  buy_units = max_buy

              else:
                  buy_units = shares
              initial_money -= buy_units * real_movement[i]
              current_inventory += buy_units
              print('day %d: buy %d units at price %f, total balance %f'% (i, buy_units, buy_units * real_movement[i], initial_money))
              states_buy.append(0)

          return initial_money, current_inventory



      #This will run for all the dates in in the df 
      for i in range(real_movement.shape[0] - int(0.025 * len(df))):
          state = signal[i]

          if state == 1:
              initial_money, current_inventory = buy(i, initial_money, current_inventory)
              states_buy.append(i)

          elif state == -1:

              if current_inventory == 0:
                      print('day %d: cannot sell anything, inventory 0' % (i))

              else:
                
                  if current_inventory > max_sell:
                      sell_units = max_sell

                  else:
                      sell_units = current_inventory
                  current_inventory -= sell_units
                  total_sell = sell_units * real_movement[i]
                  initial_money += total_sell

                  try:
                      invest = ((real_movement[i] - real_movement[states_buy[-1]])/ real_movement[states_buy[-1]]) * 100

                  except:
                      invest = 0
                  print('day %d, sell %d units at price %f, investment %f %%, total balance %f,'% (i, sell_units, total_sell, invest, initial_money))


              states_sell.append(i)



      invest = ((initial_money - starting_money) / starting_money) * 100
      total_gains = initial_money - starting_money
      return states_buy, states_sell, total_gains, invest











  states_buy, states_sell, total_gains, invest = buy_stock(df.Close, signals['positions'])







  print('total gains %f, total investment %f%%'%(total_gains, invest))


